##  patent-extractor


### Data Dictionary

df_patents (dataframe)

titles -- patent titles

dates -- issue date of patents

num_inventors -- number or inventors

num_pc -- number of patent citations

num_npc -- number of non-patent citations

num_rb -- number of references by other patents

num_class -- number of patent classifications

num_le -- number of legal events

cum_rbs -- cumulative references (sum of num_rbs)

yearly_sales

In [ ]:
# US Patents are reported by Google as html files.  Extract data from US Patents.  
# The requests library allows us to send a request to the API with the specific variables and parameters it takes
# The BeautifulSoup library allows us to parse the html formmated file we'll received after we make our request
 
import urllib2  
from bs4 import BeautifulSoup  
import numpy as np
import pandas as pd
import requests
from dateutil import parser
    
# read data into a DataFrame
df_patents = pd.read_csv('Pfizer_1.3.csv')

# Concatenate http and patent_no to find the appropriate url for google patents.  

df_patents["url"] = df_patents["http"].map(str) + df_patents["patent_no"]
del df_patents["http"]
print (df_patents.head(8))

patent_url = df_patents["url"].tolist()
patent_url = str(patent_url) 
print (len(patent_url))

patent_url = str(patent_url) 
patent_url = patent_url.replace(",","")
print (len(patent_url))


# m is counter for http string, iterate over all https 
# n is counter for filling dataframe with consecutive numbers.   

m = 0
n = 0
titles = []
dates = []
num_inventors = []
num_pcs = []
num_npcs = [] 
num_rbs = []
num_classes = []
num_les = []
issue_dates = []
num_patents = []

# Each patent url is extracted by a string of 43 characters.  m <= 430 will extract 11 patents. (m = 73,229)
while m <= 73229:
    u1 = patent_url [m+2:m+42]
    print (u1)
    patent_info = requests.get(u1)  
    m = m + 43

# Save the text that was returned into a string
# Specify the URL patent information from Google using requests.get

    content_string = patent_info.content       
    print len(content_string)
    #print (content_string) 

# Cut the patent url from the string of patents 
# Parse with BeautifulSoup, store in variable `soup`

    soup = BeautifulSoup(content_string, 'html.parser')  
    name_box = soup.find('span', attrs={"class": "main-title"})  
    name = name_box.text.strip() # strip() is used to remove starting and trailing  

# Cut the 'Patent 1234567 - ' from the name and save the title
    title = name [19:]
    print (len(titles))
    print (title)  
    titles.append(title)
    df_patents.ix[n, 'title'] = title
    
    name_box = soup.find('tr', attrs={"class": 'patent-internal-links'}) 
    events = name_box.getText()
    print(events)

# Find the publication date of the patent. 
    name_box = soup.find('table', attrs={"class": "patent-bibdata"})  
    date = name_box.getText().replace("","")

#assignee = find "Publication date"  
    d1 = date.find("Publication date")
    d2 = date.find("Filing date")
    date = date[d1+16:d2]
    dt = parser.parse(date)
    print (dt)
    print ("Publication date = " + str(date))
    dates.append(date)
    issue_dates.append(dt)
    num_patents.append(n)

# Add values to dataframe.  
    df_patents.ix[n, 'issue_date'] = dt
    df_patents.ix[n, 'date'] = date
    df_patents.ix[n, 'num_patent'] = n+620
   
    
# Extract the inventors in this box. 
    name_box = soup.find('table', attrs={"class": "patent-bibdata"}) 
    inventors = name_box.getText()

    i1 = inventors.find("Inventor") 
    i2 = inventors.find("Original Assignee")

# Inventors are contained between i1 and i2 in the string, +9 eliminates 'Inventors' from showing
    inventors = inventors[i1+9:i2]
    num_inventor = inventors.count(",") + 1 
    print ("Number of inventors = " + str(num_inventor))
    num_inventors.append(num_inventor)
    df_patents.ix[n, 'num_inventor'] = num_inventor

    
# This section extracts the 5 key values from the Google patent page.  

    name_box = soup.find('tr', attrs={"class": 'patent-internal-links'}) 
    events = (name_box.getText().replace(",",""))
    #print(events)

    a = events.count('Patent Citations') 
    b = events.count('Non-Patent Citations')
    c = events.count('Referenced by')
    d = events.count('Classifications')
    e = events.count('Legal Events')
    f = events.count('Non')
    print (a,b,c,d,e,f)
    
    
# Remove 'Patent Citations' from event string. num_pc = number of patent citations.  

    if a > 1 and f >= 1: 
        a1 = events.find("(")
        pc = events[0:a1]
        a2 = events.find(")")
        num_pc = events[a1+1:a2]
        print ("Patent Citations = " + str(num_pc))
        num_pcs.append(num_pc)
    
        # slice off 'Patent Citations' from event string. 
        events = events [a2+2:] 
    
    elif a >= 1 and f < 1: 
        a1 = events.find("(")
        pc = events[0:a1]
        a2 = events.find(")")
        num_pc = events[a1+1:a2]
        print ("Patent Citations = " + str(num_pc))
        num_pcs.append(num_pc)
    
        # slice off 'Patent Citations' from event string. 
        events = events [a2+2:]   

    elif a <= 1 and f >= 1:
        num_pc = 0
        print ("Patent Citations = " + str(num_pc)) 
        num_pcs.append(num_pc)
        
    elif a <= 0: 
        num_pc = 0
        print ("Patent Citations = " + str(num_pc)) 
        num_pcs.append(num_pc)
        
    df_patents.ix[n, 'num_pc'] = num_pc
    
# Remove 'Non-Patent Citations' from event string.  num_npc = number of non-patent citations. 
    if b > 0: 
        b1 = events.find("(")
        npc = events[0:b1]
        b2 = events.find(")")
        num_npc = events[b1+1:b2]
        print ("Non-patent citations = " + str(num_npc))
        num_npcs.append(num_npc)
    
        # slice off 'Non-Patent Citations' from event string
        events = events [b2+2:]

    elif b <= 0: 
        num_npc = 0 
        print ("Non-patent Citations = " + str(num_npc))
        num_npcs.append(num_npc)
    df_patents.ix[n, 'num_npc'] = num_npc
    
# Remove 'Referenced by' citations from event string.  num_rb = number of referenced by citations.
    if c > 0: 
        c1 = events.find("(")
        rb = events[0:c1]
        c2 = events.find(")")
        num_rb = events[c1+1:c2]
        print ("Referenced by = " + str(num_rb))
        num_rbs.append(num_rb)
    
        # slice off 'Referenced by' from event string
        events = events [c2+2:]
        
    elif c <= 0: 
        num_rb = 0 
        print ("Referenced by = " + str(num_rb))
        num_rbs.append(num_rb)
    df_patents.ix[n, 'num_rb'] = num_rb

# Remove 'Classifications' citations.  num_classif = number of classifications
    if d > 0: 
        d1 = events.find("(")
        classif = events[0:d1]
        d2 = events.find(")")
        num_class = events[d1+1:d2]
        print ("Number classifications = " + str(num_class))
        num_classes.append(num_class)
        
        # slice off 'Classifications' from event string
        events = events [d2+2:]

    elif d <= 0: 
        num_class = 0 
        print ("Number classifications = " + str(num_class)) 
        num_classes.append(num_class)
    df_patents.ix[n, 'num_class'] = num_class

# Remove 'Legal Events' citations.
    if e > 0: 
        e1 = events.find("(")
        legal = events[0:e1]
        e2 = events.find(")")
        num_le = events[e1+1:e2]
        print ("Legal events =" + str(num_le))
        num_les.append(num_le)
    
        # slice off 'Legal Events' from event string
        events = events [e2+2:]

    elif e <= 0: 
        num_le = 0 
        print ("Legal Events = " + str (num_le))
        num_les.append(num_le)
    df_patents.ix[n, 'num_le'] = num_le
    n = n + 1
    
print (titles)
print (dates)
print (num_inventors)
print (num_pcs)
print (num_npcs)
print (num_rbs)
print (num_classes)
print (num_les) 
print (issue_dates)
print (df_patents)


#### Input year and net sales data.  (Pfizer)

In [ ]:
sales = {1981: 3250, 1982: 3454, 1983: 3750, 1984: 3855, 1985: 4025, 1986: 4476, 
         1987: 4920, 1988: 5385, 1989: 5903, 1990: 6600, 1991: 7144, 1992: 7415, 
         1993: 7478, 1994: 8281, 1995: 10021, 1996: 11306, 1997: 12504, 1998: 14704, 
         1999: 16204}    

In [ ]:
# Add yearly sales to dataframe. 

yearly_sales = []

for row in df_patents.iterrows():
    row_idx = row[0]
    year = row[1]['issue_date'].year
    if not year in sales:
        continue
    fy_sales = sales[year]  
    yearly_sales.append(fy_sales)
    df_patents.loc[row_idx, 'yearly_sales'] = fy_sales
    

# Print yearly_sales and dataframe.
print df_patents.head(25)
print df_patents.tail(25)


####  Save raw data to Output.csv  

In [ ]:
df_Pfizer = df_patents
df_patents.to_csv('Output_Pfizer.csv', encoding='utf-8')

len (df_patents) 

### Drop NaN cells.  

In [ ]:
df_patents.dropna()

#### Turn string values into integers. 

In [ ]:
#  Turn string values into integers. 

df_patents['num_inventor'] = df_patents['num_inventor'].astype(int)
df_patents['num_pc'] = df_patents['num_pc'].astype(int)
df_patents['num_npc'] = df_patents['num_npc'].astype(int)
df_patents['num_rb'] = df_patents['num_rb'].astype(int)
df_patents['num_class'] = df_patents['num_class'].astype(int)
df_patents['num_le'] = df_patents['num_le'].astype(int)
df_patents['num_patent'] = df_patents['num_patent'].astype(int)
df_patents['yearly_sales'] = df_patents['yearly_sales'].astype(int)


### Diagnostics of the Pfizer data set. 

In [ ]:

print (len (num_pcs))
print (len (num_npcs))
print (len (num_rbs))
print (len (num_classes))
print (len (num_les))
print (len (num_patents))

print (len (num_patents))
print (len (yearly_sales))
print (len (issue_dates))

# Diagnostics of Pfizer dataset.  

df_patents.info()
df_patents.describe()

In [ ]:
#  Patent sum is the number of patents issued.  It starts at 430 (total no. of patents when exercise begins, in 1981).

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
sns.set()



#### Plot number of "Referenced by" plotted against "Patent Citations".  
One hypothesis is that a low no. of 'Patent Citations' and large no. of 'Referenced by' favors sales growth because the idea is novel and is widely cited.  

In [ ]:
plt.scatter(num_rbs, num_pcs)
plt.xlabel("No. of 'Referenced by' ")
plt.ylabel("Number of Patent Citations")
plt.show()


In [ ]:
# Plot the number of 'Referenced by' versus 'num_inventors'.  Best inventions have less than 5 inventors. 

plt.scatter(num_rbs, num_inventors)
plt.xlabel("No. of 'Referenced by' ")
plt.ylabel("Number of Inventors")
plt.show()


In [ ]:
# Plot no. of patents issued versus date.  Pfizer had already 430 patents at start of plot. 

import seaborn; seaborn.set()
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
sns.set()
plt.style.use('seaborn-white')


num_patents = []

value  = (df_patents['num_patent'])
num_patents.append(value)
#print value    

for row in df_patents.iterrows():      
    df_patents['num_patent'] = value 

#print (df_patents)

plt.scatter(issue_dates, num_patents, marker = 'o', color = 'black', s = 10)
plt.xlabel("Issue date of patent ")
plt.ylabel("No. of patents")



